
# Adaptive Filtering in Image Processing with OpenCV

Image filtering is an essential tool in image processing, used widely for noise reduction, 
detail enhancement, and preserving significant features like edges. In this notebook, we’ll 
dive into ***simple filtering** and *adaptive filtering** using OpenCV in Python, specifically focusing on **adaptive 
mean** ,**bilateral filtering** and **adaptive median filtering** techniques.

standard filters do an ok job in noise reduction, while not being computationally heavy, making then
an excellent choice for time sensitive applications.

Unlike standard filters, adaptive filters dynamically adjust based on the image's local 
characteristics. This adaptability makes them highly effective in images with varying noise 
levels, helping to balance noise reduction with detail preservation.


---

## Objectives of This Notebook

1. **Explore basic Filtering Concepts and effect of kernel size**: Understand how basic filters work and why they are 
   advantageous in calculation time.
2. **Explore Adaptive Filtering Concepts**: Understand how adaptive filters work and why they are 
   advantageous in noise reduction and edge preservation.
3. **Implement and Compare Filtering Techniques**: Test both adaptive and basic filters: box, gaussian and mean basic filters, as well as adaptive median, adaptive mean, and bilateral filters, we will use standarized metrics to compare them, such as **MSE**, **PSNR**, computation time and edge preservation
   filters, highlighting their differences and applications.

## Notebook Contents
- **Image Loading and Preprocessing**
- **applying noise**
- **testing basic filters**
- **Adaptive Mean Filtering**
- **Adaptive Median Filtering**
- **bilateral filtering**
- **Comparative Analysis of Filtering Results**

---

Let's begin by loading our image and performing some basic preprocessing.


we first import the necessary libraries

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import numba
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr

then we define a function that we will use alot throughout this notebook, the **display_image_with_histogram** function allows us to compare images
in a proficient way. 

In [ ]:
def display_image_with_histogram(image, title="Image", cmap='gray'):
    '''
    Displays an image and its histogram side-by-side.
    
    Parameters:
    - image: NumPy array representing the image to display.
    - title: String, title of the image display.
    - cmap: String, color map to use for displaying the image (e.g., 'gray' for grayscale images).
    '''
    # Create figure and subplots
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Display the image
    axs[0].imshow(image, cmap=cmap)
    axs[0].set_title(title)
    axs[0].axis('off')

    # Display histogram
    if len(image.shape) == 2:  # Grayscale image
        axs[1].hist(image.ravel(), bins=256, color='gray', alpha=0.7)
        axs[1].set_title("Histogram (Grayscale)")
        axs[1].set_xlabel("Pixel Intensity")
        axs[1].set_ylabel("Frequency")
    else:  # Color image
        colors = ('b', 'g', 'r')
        for i, color in enumerate(colors):
            axs[1].hist(image[:, :, i].ravel(), bins=256, color=color, alpha=0.7, label=f'{color.upper()} Channel')
        axs[1].set_title("Histogram (Color)")
        axs[1].set_xlabel("Pixel Intensity")
        axs[1].set_ylabel("Frequency")
        axs[1].legend()

    # Adjust layout and show
    plt.tight_layout()
    plt.show()

## 1. Image Loading and Preprocessing

In this first step, we'll load a sample image in **RGB format**. Using RGB images allows us to observe how filtering techniques affect each color channel independently and jointly, providing a more comprehensive view of the filter’s impact on color images.

We'll use OpenCV's `cv2.imread()` function to load the image in color mode. This approach is useful for applications where maintaining color integrity across channels is important, as filtering operations can impact each channel differently.

In [ ]:
#make dictionaries that will be filled later
images_dict = {}
noisy_images_dict = {}
box_filter_dict = {}
gaussian_filter_dict = {}
median_filter_dict = {}
adaptive_median_filter_dict = {}
adaptive_mean_filter_dictt = {}
bilateral_filter_dict = {}


In [ ]:
#here we will store the photos
path = 'images'
#just some single liners
images = [i for i in os.listdir(path) if i.endswith('.jpg') or i.endswith('.png')]
images_rgb = [cv.cvtColor(cv.imread(f'{path}/{i}'), cv.COLOR_BGR2RGB) for i in images]
images_np = [np.array(i) for i in images_rgb]
images_dict["image_names"] = images
images_dict["images_rgb"] = images_rgb
images_dict["images_np"] = images_np

In [ ]:
#this cell os for image display only
for idx, img in enumerate(images_np):
    display_image_with_histogram(img, title=images[idx])


## 2. Applying noise

To evaluate the effectiveness of filtering techniques, we first need to simulate noisy conditions by adding noise to our image. Introducing noise allows us to assess how well each filter can reduce or eliminate unwanted disturbances without overly blurring important details.

### Types of Noise Added:
1. **Gaussian Noise**:
   - **Description**: Gaussian noise is continuous and follows a normal distribution, where the noise has random values centered around a mean with some variance. This noise type is commonly encountered in real-world scenarios, and is considered a type of additive noise, usually gaussian filters ge trid of gaussian noise pretty well, and taht is what we are going to try out.
   
   - **Purpose**: Gaussian noise provides a challenging test case for filters, as it affects every pixel in the image. Testing filters on Gaussian noise helps determine their ability to smooth continuous disturbances and maintain image quality.

2. **Salt-and-Pepper Noise**:
   - **Description**: Salt-and-pepper noise introduces random black (pepper) and white (salt) pixels throughout the image, creating a "speckled" effect. This noise often results from bit errors in image transmission or low-light environments.
   
   - **Purpose**: Salt-and-pepper noise is particularly challenging for filters because it creates stark, isolated pixel variations. Filters need to remove these extreme outliers effectively without blurring surrounding areas, making it a strong test for adaptive filtering methods, like the median filter, which handles this type of noise effectively.

By applying both Gaussian and salt-and-pepper noise, we can examine the versatility of each filtering method and how well they address different noise characteristics.

the **apply_noise** function described below utilizes the numpy library to generate both types of noises, below we used `np.random` to randomize the noise


In [ ]:

def apply_noise(noise_type, image, intensity):
    '''
    Adds salt-and-pepper noise or Gaussian noise to a color image.    
    Parameters:
    - noise_type: String, type of noise to be applied. Can be 'salt_pepper' or 'gaussian'.
    - image: Color (RGB) image as a NumPy array.
    - intensity: Float, controls the noise level. For 'salt_pepper', it's the percentage of pixels to be changed.
                 For 'gaussian', it scales the variance of the noise.
    
    Returns:
    - Noisy image with the specified noise applied.
    '''
    row, col, ch = image.shape
    out = np.copy(image)

    if noise_type == 'salt_pepper':
        # Calculate number of salt and pepper pixels
        num_salt = int(np.ceil(intensity * image.size * 0.5))
        num_pepper = int(np.ceil(intensity * image.size * 0.5))
        
        # Generate salt noise (set pixels to [255, 255, 255] for all channels)
        salt_coords = (np.random.randint(0, row, num_salt), np.random.randint(0, col, num_salt))
        out[salt_coords] = [255] * ch  # Applies 255 to all channels at specified coordinates
        
        # Generate pepper noise (set pixels to [0, 0, 0] for all channels)
        pepper_coords = (np.random.randint(0, row, num_pepper), np.random.randint(0, col, num_pepper))
        out[pepper_coords] = [0] * ch  # Applies 0 to all channels at specified coordinates

    elif noise_type == 'gaussian':
        # Apply Gaussian noise based on intensity
        mean = 0
        var = intensity  # Set variance based on intensity
        sigma = var ** 0.5
        gauss = np.random.normal(mean, sigma, (row, col, ch))
        
        # Add Gaussian noise to the image and clip to valid range
        out = image + gauss * 255
        out = np.clip(out, 0, 255).astype(np.uint8)  # Ensure values are within valid range for display

    else:
        raise ValueError('Invalid noise type. Please use "salt_pepper" or "gaussian".')

    return out


### Applying Salt-and-Pepper Noise

Below, we generate and display the images with added salt-and-pepper noise, and we store the images in `sp_noised_imgs` list.

In [ ]:
noisy_images_dict["low_sp_noised_images"] = [apply_noise('salt_pepper', img, 0.01) for img in images_np]
noisy_images_dict["med_sp_noised_images"] = [apply_noise('salt_pepper', img, 0.05) for img in images_np]
noisy_images_dict["high_sp_noised_images"] = [apply_noise('salt_pepper', img, 0.1) for img in images_np]


in the above cell, we applied the two types of noise with varying intensities, and i tried to store them in a matter that is easy to access again, where the storage is a well divided hierarical dictionary.

In [ ]:
#this cell is solely for displaying noised images.
for idx, img in enumerate(noisy_images_dict["low_sp_noised_images"]):
    display_image_with_histogram(noisy_images_dict["low_sp_noised_images"][idx], title=f'{images[idx]} (Low Salt-and-Pepper Noise)')
    display_image_with_histogram(noisy_images_dict["med_sp_noised_images"][idx], title=f'{images[idx]} (Medium Salt-and-Pepper Noise)')
    display_image_with_histogram(noisy_images_dict["high_sp_noised_images"][idx], title=f'{images[idx]} (High Salt-and-Pepper Noise)')

### Applying guassian Noise

Below, we generate and display the images with added gaussian noise, and we store the images in `gaussian_noised_imgs` list.

store all the images, and plot them in the next 2 cells

In [ ]:
noisy_images_dict["low_gauss_noised_images"] = [apply_noise('gaussian', img, 0.01) for img in images_np]
noisy_images_dict["med_gauss_noised_images"] = [apply_noise('gaussian', img, 0.08) for img in images_np]
noisy_images_dict["high_gauss_noised_images"] = [apply_noise('gaussian', img, 0.2) for img in images_np]

In [ ]:
#display the noised images
for idx, img in enumerate(noisy_images_dict["low_gauss_noised_images"]):
    display_image_with_histogram(noisy_images_dict["low_gauss_noised_images"][idx], title=f'{images[idx]} (Low Gaussian Noise)')
    display_image_with_histogram(noisy_images_dict["med_gauss_noised_images"][idx], title=f'{images[idx]} (Medium Gaussian Noise)')
    display_image_with_histogram(noisy_images_dict["high_gauss_noised_images"][idx], title=f'{images[idx]} (High Gaussian Noise)')

In [ ]:
#we will now store the images
output_dir = "noisey_images"
os.makedirs(output_dir, exist_ok=True)
# Iterate over noise levels, noise types, and images
for noise_level in ['low', 'med', 'high']:
    for noise_type in ['sp', 'gauss']:
        for idx, img in enumerate(images_np):
            # Create the subdirectory path for each image, noise level, and type
            img_name = images[idx]  # assuming 'images' contains image names
            subdir_path = os.path.join(output_dir, img_name, noise_level)
            os.makedirs(subdir_path, exist_ok=True)
            
            # Save the noisy image in the appropriate subdirectory
            output_path = os.path.join(subdir_path, f'{noise_type}_noise.png')
            noisy_image = noisy_images_dict[f'{noise_level}_{noise_type}_noised_images'][idx]
            cv.imwrite(output_path, cv.cvtColor(noisy_image, cv.COLOR_RGB2BGR))


## 3. Basic Filtering

Basic image filters are foundational in image processing, often applied to reduce noise or blur specific details to achieve desired effects. This section covers three commonly used filters—**Box Filter**, **Gaussian Filter**, and **Median Filter**—each serving different purposes in image smoothing.

### Types of Filters

1. **Box Filter**:
   - This filter replaces each pixel's value with the average value of the pixels in a surrounding box-shaped neighborhood (kernel).
   - It provides a simple blur effect, softening edges and reducing fine details, but is less effective at preserving sharp edges.

2. **Gaussian Filter**:
   - Gaussian filtering uses a kernel that follows a Gaussian distribution, applying a weighted average where pixels closer to the center contribute more heavily.
   - This filter produces a more natural blur and is commonly used for noise reduction, offering a gentler smoothing effect than the box filter.

3. **Median Filter**:
   - The median filter replaces each pixel with the median value of the pixels within its kernel, making it especially effective at removing salt-and-pepper noise.
   - This filter preserves edges better than averaging filters since it reduces extreme pixel values without spreading them.

### `apply_simple_filters` Function

The `apply_simple_filters` function applies one of these three filters to an image, with options for both grayscale and color images. Key parameters in the function are:
- `filter_type`: Specifies the filter to apply (`'box filter'`, `'gaussian filter'`, or `'median filter'`).
- `colored`: Indicates whether the image is colored or grayscale. When colored, each channel (Blue, Green, Red) is processed separately.
- `kernel_size`: Defines the size of the kernel (odd integer) for the filtering operation. Larger kernel sizes increase the level of smoothing.

The function processes each color channel individually if the image is colored, ensuring the filter is applied to each channel consistently. It then merges the filtered channels back together for the final output. This flexibility makes it a versatile tool for basic image smoothing tasks in color or grayscale formats.

# Mathematics of Box and Gaussian Image Processing Filters

Image filters are fundamental tools in image processing used to enhance or modify an image. Two widely used filters are the **Box filter** and the **Gaussian filter**. Both are linear filters applied to an image using a convolution operation.

---

## 1. The Box Filter

### Definition
The **Box filter** is a simple averaging filter. It smoothens an image by replacing each pixel's value with the average value of its neighboring pixels, determined by the filter's kernel size.

### Mathematical Representation
Given an image \( I(x, y) \) and a square kernel of size \( k \times k \), the output image \( O(x, y) \) after applying a Box filter is:

$$
O(x, y) = \frac{1}{k^2} \sum_{i=-\frac{k}{2}}^{\frac{k}{2}} \sum_{j=-\frac{k}{2}}^{\frac{k}{2}} I(x+i, y+j)
$$

Where:
- \( k \) is the kernel size (e.g., 3x3, 5x5).
- \( \frac{1}{k^2} \) ensures that the kernel weights sum to 1, preserving image brightness.

### Kernel
The kernel for a Box filter is a matrix where all elements are equal to \( \frac{1}{k^2} \). For example, a 3x3 Box filter kernel is:

$$
K = \frac{1}{9} \begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

### Characteristics
- Computationally efficient.
- Causes significant blurring.
- Introduces edge artifacts due to uniform weighting.

---

## 2. The Gaussian Filter

### Definition
The **Gaussian filter** is a smoothing filter that uses a Gaussian function to assign weights to neighboring pixels. It is effective at reducing noise and preserving edges compared to the Box filter.

### Gaussian Function
The Gaussian function in 2D is defined as:

$$
G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

Where:
- \( \sigma \) is the standard deviation, controlling the filter's spread.

### Mathematical Representation
The output image \( O(x, y) \) is obtained by convolving the Gaussian kernel \( G(x, y) \) with the input image \( I(x, y) \):

$$
O(x, y) = \sum_{i=-\frac{k}{2}}^{\frac{k}{2}} \sum_{j=-\frac{k}{2}}^{\frac{k}{2}} G(i, j) \cdot I(x+i, y+j)
$$

Where \( G(i, j) \) is the weight derived from the Gaussian function.

### Kernel
The Gaussian kernel values are derived from the Gaussian function. For instance, a 3x3 kernel with sigma = 1 might look like:

$$
K = \frac{1}{16} \begin{bmatrix}
1 & 2 & 1 \\
2 & 4 & 2 \\
1 & 2 & 1
\end{bmatrix}
$$

### Characteristics
- Weights decay with distance, reducing edge artifacts.
- More computationally intensive than the Box filter.
- Effective for noise reduction while preserving image details.

---

## 3. Comparison of Box and Gaussian Filters

| Feature                | Box Filter                          | Gaussian Filter                     |
|------------------------|-------------------------------------|-------------------------------------|
| **Kernel Weights**     | Uniform (all equal)                | Vary based on Gaussian function     |
| **Blur Type**          | Uniform                            | Weighted (center-weighted)          |
| **Edge Artifacts**     | Higher                             | Lower                               |
| **Noise Reduction**    | Less effective                     | More effective                      |
| **Computational Cost** | Lower                              | Higher                              |


In [ ]:
def apply_simple_filters(image, filter_type='box filter', colored=True, kernel_size=5):
    '''
    Applies simple filters to a grayscale or colored image.

    Parameters:
    - image: NumPy array representing the image to apply filters on.
    - filter_type: String, type of filter to be applied. Can be 'box filter', 'gaussian filter', or 'median filter'.
    - colored: Boolean, whether the image is colored (True) or grayscale (False).
    - kernel_size: Integer, size of the kernel for the filter. Should be an odd number.
    
    Returns:
    - filtered_image: Filtered image as a NumPy array.
    '''
    if kernel_size % 2 == 0:
        # Kernel size must be odd,since there is no center pixel in an even kernel
        raise ValueError("kernel_size must be an odd number.")

    if colored:
        # Split the image into its color channels
        b, g, r = cv.split(image)

        # Apply the filter to each channel
        if filter_type == 'box filter':
            b_filtered = cv.blur(b, (kernel_size, kernel_size))
            g_filtered = cv.blur(g, (kernel_size, kernel_size))
            r_filtered = cv.blur(r, (kernel_size, kernel_size))
        elif filter_type == 'gaussian filter':
            b_filtered = cv.GaussianBlur(b, (kernel_size, kernel_size), 0)# 0 for the standard deviation, it will be calculated automatically
            g_filtered = cv.GaussianBlur(g, (kernel_size, kernel_size), 0)
            r_filtered = cv.GaussianBlur(r, (kernel_size, kernel_size), 0)
        elif filter_type == 'median filter':
            #median filter takes kernel size as an integer, because it is a square kernel necessarily
            b_filtered = cv.medianBlur(b, kernel_size)
            g_filtered = cv.medianBlur(g, kernel_size)
            r_filtered = cv.medianBlur(r, kernel_size)
        else:
            raise ValueError('Invalid filter type. Please use "box filter", "gaussian filter", or "median filter".')

        # Merge the filtered channels back into an image
        filtered_image = cv.merge((b_filtered, g_filtered, r_filtered))
    else:
        # Apply the filter to the grayscale image
        if filter_type == 'box filter':
            filtered_image = cv.blur(image, (kernel_size, kernel_size))
        elif filter_type == 'gaussian filter':
            filtered_image = cv.GaussianBlur(image, (kernel_size, kernel_size), 0)
        elif filter_type == 'median filter':
            filtered_image = cv.medianBlur(image, kernel_size)
        else:
            raise ValueError('Invalid filter type. Please use "box filter", "gaussian filter", or "median filter".')

    return filtered_image


In [ ]:
def save_filtered_images(filter_dict, noisy_images_dict, images, output_dir, filter_name):
    """
    Save filtered images in a nested directory structure based on noise level and kernel size.
    
    Parameters:
    - filter_dict: Dictionary containing filtered images organized by noise level and kernel size.
    - noisy_images_dict: Dictionary containing noisy images organized by noise level.
    - images: List of image names.
    - output_dir: Main directory to save the filtered images.
    - filter_name: Name of the filter for file naming.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for idx, img_name in enumerate(images):
        for noise_level, filtered_imgs in filter_dict.items():
            for size_label, img_list in filtered_imgs.items():
                # Create subdirectory path for each image, noise level, and kernel size
                subdir_path = os.path.join(output_dir, img_name, noise_level)
                os.makedirs(subdir_path, exist_ok=True)
                
                # Save the filtered image with appropriate naming
                output_path = os.path.join(subdir_path, f"{size_label}_{filter_name}.png")
                cv.imwrite(output_path, cv.cvtColor(img_list[idx], cv.COLOR_RGB2BGR))

In [ ]:
def display_filtered_images_with_histogram(filter_dict, noisy_images_dict, images, filter_name):
    """
    Display noisy and filtered images with histograms for visual comparison.
    
    Parameters:
    - filter_dict: Dictionary containing filtered images organized by noise level and kernel size.
    - noisy_images_dict: Dictionary containing noisy images organized by noise level.
    - images: List of image names.
    - filter_name: Name of the filter for display titles.
    """
    for idx, img_name in enumerate(images):
        for noise_level, filtered_imgs in filter_dict.items():
            for size_label, img_list in filtered_imgs.items():
                # Set up a side-by-side layout for noisy and filtered images
                fig, axs = plt.subplots(2, 2, figsize=(10, 8))
                fig.suptitle(f'{img_name} - {noise_level} Noise - {size_label} Kernel - {filter_name}', fontsize=14)

                # Display noisy image and its histogram
                noisy_image = noisy_images_dict[noise_level][idx]
                axs[0, 0].imshow(noisy_image)
                axs[0, 0].set_title('Noisy Image')
                axs[0, 0].axis('off')
                
                axs[1, 0].hist(noisy_image.ravel(), bins=256, color='gray')
                axs[1, 0].set_title('Noisy Image Histogram')
                
                # Display filtered image and its histogram
                filtered_image = img_list[idx]
                axs[0, 1].imshow(filtered_image)
                axs[0, 1].set_title(f'Filtered Image ({filter_name})')
                axs[0, 1].axis('off')
                
                axs[1, 1].hist(filtered_image.ravel(), bins=256, color='gray')
                axs[1, 1].set_title('Filtered Image Histogram')
                
                plt.tight_layout(rect=[0, 0, 1, 0.96])
                plt.show()


### Applying Basic Filters to Salt-and-Pepper/gaussian Noised Images

Now, we apply box, Gaussian, and median filters to the salt-and-pepper/gaussian noised images to observe each filter’s effect on noise reduction.

In [ ]:
# Define kernel sizes

kernel_sizes = {'small': 3, 'medium': 5, 'big': 7}

# Define the main output directory for box-filtered images
output_dir = 'box_filtered_images'
os.makedirs(output_dir, exist_ok=True)


box_filter_dict = {
    "low_sp_noised_images": {},
    "med_sp_noised_images": {},
    "high_sp_noised_images": {},
    "low_gauss_noised_images": {},
    "med_gauss_noised_images": {},
    "high_gauss_noised_images": {}
}


in the below cell we will use the `%%timeit` to get the average run time for the below cell, which consists of using the 

In [ ]:
%%timeit

# Apply box filters with varying kernel sizes for each noise level
for noise_level in ["low_sp_noised_images", "med_sp_noised_images", "high_sp_noised_images", "low_gauss_noised_images", "med_gauss_noised_images", "high_gauss_noised_images"]:
    for size_label, k_size in kernel_sizes.items():
        # Apply box filter to each noisy image and store in the dictionary
        box_filter_dict[noise_level][size_label] = [
            apply_simple_filters(image, filter_type="box filter",kernel_size=k_size, colored=True) for image in noisy_images_dict[noise_level]
        ]


In [ ]:
#save the images
save_filtered_images(box_filter_dict, noisy_images_dict, images, output_dir, "box_filter")

In [ ]:
display_filtered_images_with_histogram(box_filter_dict, noisy_images_dict, images, "Box Filter")

In [ ]:
# Define kernel sizes

kernel_sizes = {'small': 3, 'medium': 5, 'big': 7}

gaussian_filter_dict = {
    "low_sp_noised_images": {},
    "med_sp_noised_images": {},
    "high_sp_noised_images": {},
    "low_gauss_noised_images": {},
    "med_gauss_noised_images": {},
    "high_gauss_noised_images": {}
}


it is important to note that the openCV library calculates the sigma value automatically based on the kernel size, so there is no need to expirement with it, as it only complicates the computational permutations.

In [ ]:
%%timeit
# Apply gaussian filters with varying kernel sizes for each noise level

for noise_level in ["low_sp_noised_images", "med_sp_noised_images", "high_sp_noised_images", "low_gauss_noised_images", "med_gauss_noised_images", "high_gauss_noised_images"]:
    for size_label, k_size in kernel_sizes.items():
        # Apply box filter to each noisy image and store in the dictionary
        gaussian_filter_dict[noise_level][size_label] = [
            apply_simple_filters(image, filter_type="gaussian filter",kernel_size=k_size, colored=True) for image in noisy_images_dict[noise_level]
        ]

In [ ]:
# Define the main output directory for box-filtered images
output_dir = 'gaussian_filtered_images'
os.makedirs(output_dir, exist_ok=True)
#save the images
save_filtered_images(box_filter_dict, noisy_images_dict, images, output_dir, "gaussian_filter")

In [ ]:
display_filtered_images_with_histogram(gaussian_filter_dict, noisy_images_dict, images, "Gaussian Filter")

In [ ]:
# Define kernel sizes
kernel_sizes = {'small': 3, 'medium': 5, 'big': 7}

median_filter_dict = {
    "low_sp_noised_images": {},
    "med_sp_noised_images": {},
    "high_sp_noised_images": {},
    "low_gauss_noised_images": {},
    "med_gauss_noised_images": {},
    "high_gauss_noised_images": {}
}


In [ ]:
%%timeit
# Apply box filters with varying kernel sizes for each noise level
for noise_level in ["low_sp_noised_images", "med_sp_noised_images", "high_sp_noised_images", "low_gauss_noised_images", "med_gauss_noised_images", "high_gauss_noised_images"]:
    for size_label, k_size in kernel_sizes.items():
        # Apply median filter to each noisy image and store in the dictionary
        median_filter_dict[noise_level][size_label] = [
            apply_simple_filters(image, filter_type="median filter",kernel_size=k_size, colored=True) for image in noisy_images_dict[noise_level]
        ]

In [ ]:
# Define the main output directory for median-filtered images
output_dir = 'median_filtered_images'
os.makedirs(output_dir, exist_ok=True)

save_filtered_images(median_filter_dict, noisy_images_dict, images, output_dir, "median_filter")

In [ ]:
# Display filtered images with histograms
display_filtered_images_with_histogram(median_filter_dict, noisy_images_dict, images, "Median Filter")

we note the difference in performance between the 3 classes of filters, while the gaussian and box filter took almost 50 ms on average 

# Comparison of Filter Performances

The performance of three commonly used image processing filters—Median Filter, Gaussian Filter, and Box Filter—was evaluated based on their execution times. The results below highlight the efficiency of each filter.

---

## **Execution Times**

| Filter Type       | Execution Time (Mean ± Std. Dev.) | Loops per Run |
|-------------------|-----------------------------------|---------------|
| **Median Filter** | 496 ms ± 12.8 ms                 | 1 loop        |
| **Gaussian Filter**| 43.5 ms ± 10.2 ms               | 10 loops      |
| **Box Filter**    | 35.4 ms ± 1.83 ms                | 10 loops      |

---

## **Key Observations**
1. **Median Filter**:
   - **Execution Time**: The slowest filter with an average execution time of **496 ms** per loop.
   - **Reason**: The Median Filter requires sorting neighboring pixels to compute the median, which is computationally intensive, especially for large kernels.

2. **Gaussian Filter**:
   - **Execution Time**: Significantly faster than the Median Filter, averaging **43.5 ms** per loop.
   - **Reason**: The Gaussian Filter uses a weighted sum of neighboring pixels, which is computationally efficient and benefits from optimized mathematical operations.

3. **Box Filter**:
   - **Execution Time**: The fastest filter, averaging **35.4 ms** per loop.
   - **Reason**: The Box Filter applies a uniform averaging operation, which involves simple addition and division, making it highly efficient.

---

## **Performance Ranking**
1. **Box Filter**: Most efficient filter with the lowest execution time.
2. **Gaussian Filter**: Moderately efficient and significantly faster than the Median Filter.
3. **Median Filter**: Least efficient due to its computational complexity.

---

## **Choosing the Right Filter**
- **Median Filter**:
  - **Best For**: Reducing noise like "salt-and-pepper" while preserving edges.
  - **Trade-off**: High computational cost.

- **Gaussian Filter**:
  - **Best For**: Smoothing images while maintaining some detail, with better performance than the Median Filter.
  - **Trade-off**: Slightly blurs edges.

- **Box Filter**:
  - **Best For**: Quick and simple smoothing tasks.
  - **Trade-off**: May introduce artifacts due to uniform averaging.

---

## **Conclusion**
The choice of filter depends on the application and trade-offs between computational efficiency and the quality of image processing. For real-time applications, the **Box Filter** is the best option, while the **Median Filter** is ideal for applications requiring noise reduction with edge preservation, despite its slower performance.


## Objective metrics(MSE, SQNR) of images for basic features

In [ ]:
def compute_metrics_by_noise_level(original_images, filter_dict):
    """
    Calculate MSE and SQNR for each kernel size and noise level, and return metrics for each combination.
    
    Parameters:
        original_images (list): List of original images (ground truth).
        filter_dict (dict): Dictionary containing filtered images organized as
                            filter_dict[noise_level][kernel_size].
    
    Returns:
        filter_metrics (dict): Dictionary with MSE and SQNR for each kernel size and noise level.
                               Structure: filter_metrics[noise_level][kernel_size][metric]
    """
    filter_metrics = {}

    for noise_level in filter_dict:  # Iterate over noise levels
        filter_metrics[noise_level] = {}

        for kernel_size in filter_dict[noise_level]:  # Iterate over kernel sizes for the current noise level
            total_mse = 0
            total_sqnr = 0
            total_images = 0

            filtered_images = filter_dict[noise_level][kernel_size]
            for i, filtered_image in enumerate(filtered_images):
                mse_val = mse(original_images[i], filtered_image)
                sqnr_val = psnr(original_images[i], filtered_image)
                total_mse += mse_val
                total_sqnr += sqnr_val
                total_images += 1

            # Calculate averages for the current kernel size and noise level
            avg_mse = total_mse / total_images
            avg_sqnr = total_sqnr / total_images

            # Store results in the dictionary
            filter_metrics[noise_level][kernel_size] = {"MSE": avg_mse, "SQNR": avg_sqnr}

    return filter_metrics


In [ ]:
def compute_metrics_without_kernel(original_images, noise_dict):
    """
    Calculate average MSE and SQNR for each noise level across all images.
    
    Parameters:
        original_images (list): List of original images (ground truth).
        noise_dict (dict): Dictionary containing filtered images organized as
                           noise_dict[noise_level].
    
    Returns:
        noise_metrics (dict): Dictionary with average MSE and SQNR for each noise level.
                               Structure: noise_metrics[noise_level][metric]
    """
    noise_metrics = {}

    for noise_level in noise_dict:  # Iterate over noise levels
        total_mse = 0
        total_sqnr = 0
        total_images = 0

        filtered_images = noise_dict[noise_level]
        for i, filtered_image in enumerate(filtered_images):
            mse_val = mse(original_images[i], filtered_image)
            sqnr_val = psnr(original_images[i], filtered_image)
            total_mse += mse_val
            total_sqnr += sqnr_val
            total_images += 1

        # Calculate averages for the current noise level
        avg_mse = total_mse / total_images
        avg_sqnr = total_sqnr / total_images

        # Store results in the dictionary
        noise_metrics[noise_level] = {"MSE": avg_mse, "SQNR": avg_sqnr}

    return noise_metrics


In [ ]:
box_filter_metrics = compute_metrics_by_noise_level(images_np, box_filter_dict)
gaussian_filter_metrics = compute_metrics_by_noise_level(images_np, gaussian_filter_dict)
median_filter_metrics = compute_metrics_by_noise_level(images_np, median_filter_dict)

noised_images_metrics = compute_metrics_without_kernel(images_np, noisy_images_dict)

#also compare with the noised images

#we will apply the metric calculation versus the noised images


In [ ]:
#print the results
print("Box Filter Metrics:")
for noise_level, kernel_metrics in box_filter_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    for kernel_size, metrics in kernel_metrics.items():
        print(f"Kernel Size: {kernel_size}")
        print(f"MSE: {metrics['MSE']:.2f}")
        print(f"SQNR: {metrics['SQNR']:.2f}")

print("\nGaussian Filter Metrics:")
for noise_level, kernel_metrics in gaussian_filter_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    for kernel_size, metrics in kernel_metrics.items():
        print(f"Kernel Size: {kernel_size}")
        print(f"MSE: {metrics['MSE']:.2f}")
        print(f"SQNR: {metrics['SQNR']:.2f}")

print("\nMedian Filter Metrics:")
for noise_level, kernel_metrics in median_filter_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    for kernel_size, metrics in kernel_metrics.items():
        print(f"Kernel Size: {kernel_size}")
        print(f"MSE: {metrics['MSE']:.2f}")
        print(f"SQNR: {metrics['SQNR']:.2f}")

print("\nNoised Images Metrics:")
for noise_level, metrics in noised_images_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    print(f"MSE: {metrics['MSE']:.2f}")
    print(f"SQNR: {metrics['SQNR']:.2f}")

In [ ]:
 list(box_filter_metrics.keys())

In [ ]:
def plot_filter_metrics_with_noise_level(box_filter_metrics, gaussian_filter_metrics, median_filter_metrics):
    """
    Plot histograms of SQNR and MSE for different kernel sizes, filter types, and noise levels.
    
    Parameters:
    - box_filter_metrics (dict): Dictionary containing MSE and SQNR for Box Filter with different kernel sizes and noise levels.
    - gaussian_filter_metrics (dict): Dictionary containing MSE and SQNR for Gaussian Filter with different kernel sizes and noise levels.
    - median_filter_metrics (dict): Dictionary containing MSE and SQNR for Median Filter with different kernel sizes and noise levels.
    """
    # Prepare the data for histograms
    noise_levels = list(box_filter_metrics.keys())  # Assume all dictionaries have the same noise levels
    kernel_sizes = list(box_filter_metrics[noise_levels[0]].keys())  # Assume all dictionaries have the same kernel sizes

    # Create a grid of subplots: one for MSE and one for SQNR for each noise level and kernel size
    num_plots = len(noise_levels) * len(kernel_sizes)
    fig, axs = plt.subplots(num_plots, 2, figsize=(12, num_plots * 4))  # 2 columns (MSE, SQNR) for each plot

    # Loop through the noise levels and kernel sizes and plot for each
    plot_idx = 0
    for noise_level in noise_levels:
        for kernel_size in kernel_sizes:
            # Get MSE and SQNR for the Box, Gaussian, and Median Filters
            box_mse = box_filter_metrics[noise_level][kernel_size]["MSE"]
            box_sqnr = box_filter_metrics[noise_level][kernel_size]["SQNR"]
            
            gaussian_mse = gaussian_filter_metrics[noise_level][kernel_size]["MSE"]
            gaussian_sqnr = gaussian_filter_metrics[noise_level][kernel_size]["SQNR"]
            
            median_mse = median_filter_metrics[noise_level][kernel_size]["MSE"]
            median_sqnr = median_filter_metrics[noise_level][kernel_size]["SQNR"]
            
            # Prepare data for plotting
            mse_values = [box_mse, gaussian_mse, median_mse]
            sqnr_values = [box_sqnr, gaussian_sqnr, median_sqnr]
            filters = ['Box Filter', 'Gaussian Filter', 'Median Filter']
            
            # Plot MSE for current noise level and kernel size
            axs[plot_idx, 0].bar(filters, mse_values, color=['blue', 'green', 'red'])
            axs[plot_idx, 0].set_title(f'{kernel_size}, {noise_level}')
            axs[plot_idx, 0].set_ylabel('MSE')
            
            # Plot SQNR for current noise level and kernel size
            axs[plot_idx, 1].bar(filters, sqnr_values, color=['blue', 'green', 'red'])
            axs[plot_idx, 1].set_title(f'{kernel_size}, {noise_level}')
            axs[plot_idx, 1].set_ylabel('SQNR')
            
            plot_idx += 1  # Move to the next plot

    # Adjust layout to prevent overlapping
    plt.tight_layout()
    plt.show()


In [ ]:
plot_filter_metrics_with_noise_level(box_filter_metrics, gaussian_filter_metrics, median_filter_metrics)

we note that for the salt and pepper noise in general, the median filter excels over box and gaussian, with an average higher mse, and an average higher SQNR, it justifies the long time it relatively takes to run, and we also notice that in the gaussian noised images, the gaussian filter came close to the median filter, that means that it is not a bad choise for getting rid of gaussian noise.


## 4. Adaptive Mean Filtering

### What is Adaptive Mean Filtering?

Adaptive Mean Filtering calculates the average of pixel values in a neighborhood around each pixel. 
However, unlike traditional mean filtering, the kernel size or the filter’s effect adjusts based 
on local variance within the region. This dynamic approach helps reduce noise while preserving edge 
details.

### Key Characteristics

- **Smooth Areas**: In regions with low variance, a larger kernel can be used to average values and 
  reduce noise effectively.
- **Edge Regions**: In areas with high variance (like edges), a smaller kernel size is often preferred 
  to avoid blurring important features.

In the code below, we apply an adaptive mean filter that adapts the kernel size based on the local 
image variance, providing a more context-aware filtering solution, and since opencv doesn't have a
direct implementation, below is my implementation for the adaptive mean filtering algorithm.

In [ ]:
def adaptive_mean_filtering(image, max_kernel_size=7, variance_threshold=10):
    """
    Apply adaptive mean filtering on an RGB or grayscale image, with an adaptive kernel size based on local variance.
    
    Parameters:
    - image: Input image in RGB or grayscale format.
    - max_kernel_size: Maximum kernel size (must be an odd integer).
    - variance_threshold: Variance threshold for controlling kernel size growth.
    
    Returns:
    - Filtered image.
    """
    if max_kernel_size % 2 == 0:
        raise ValueError("max_kernel_size must be an odd integer.")
    
    if len(image.shape) == 2:  # Grayscale image
        return _adaptive_mean_filter_single_channel(image, max_kernel_size, variance_threshold)
    elif len(image.shape) == 3:  # RGB image
        channels = cv.split(image)
        filtered_channels = [ _adaptive_mean_filter_single_channel(ch, max_kernel_size, variance_threshold) for ch in channels]
        return cv.merge(filtered_channels)
    else:
        raise ValueError("Input image must be either grayscale or RGB.")

def _adaptive_mean_filter_single_channel(channel, max_kernel_size, variance_threshold):
    """
    Apply adaptive mean filtering on a single-channel (grayscale) image.
    
    Parameters:
    - channel: Single-channel (grayscale) image.
    - max_kernel_size: Maximum kernel size (must be an odd integer).
    - variance_threshold: Variance threshold for controlling kernel size growth.
    
    Returns:
    - Filtered single-channel image.
    """
    channel = channel.astype(np.float32)
    filtered_image = np.zeros_like(channel)
    pad_size = max_kernel_size // 2
    padded_image = cv.copyMakeBorder(channel, pad_size, pad_size, pad_size, pad_size, cv.BORDER_REFLECT)
    
    for i in range(channel.shape[0]):
        for j in range(channel.shape[1]):
            kernel_size = 3  # Start with the smallest kernel size
            while kernel_size <= max_kernel_size:
                half_k = kernel_size // 2
                # Extract the local region around the current pixel
                kernel_region = padded_image[i:i + kernel_size, j:j + kernel_size]

                # Calculate local mean and variance
                local_mean = np.mean(kernel_region)
                local_variance = np.var(kernel_region)
                
                # Assign mean if variance is below threshold or if max kernel size is reached
                if local_variance < variance_threshold or kernel_size == max_kernel_size:
                    filtered_image[i, j] = local_mean
                    break
                else:
                    kernel_size += 2  # Increase kernel size, keeping it odd
                
    return np.clip(filtered_image, 0, 255).astype(np.uint8)


In [ ]:
adaptive_mean_res = adaptive_mean_filtering(noisy_images_dict["med_sp_noised_images"][2], max_kernel_size=7, variance_threshold=10)

# Display the original and filtered images
display_image_with_histogram(adaptive_mean_res, title="Adaptive Mean Filtered Image")

In [ ]:
#apply the adaptive mean filtering for salt pepper
adaptive_mean_filter_dict = {
    "low_sp_noised_images": {},
    "med_sp_noised_images": {},
    "high_sp_noised_images": {},
    "low_gauss_noised_images": {},
    "med_gauss_noised_images": {},
    "high_gauss_noised_images": {}
}

for noise_level in ["low_sp_noised_images", "med_sp_noised_images", "high_sp_noised_images", "low_gauss_noised_images", "med_gauss_noised_images", "high_gauss_noised_images"]:
    for idx, img in enumerate(noisy_images_dict[noise_level]):
        adaptive_mean_filter_dict[noise_level][idx] = adaptive_mean_filtering(img, max_kernel_size=7, variance_threshold=16)

In [ ]:
display_image_with_histogram(adaptive_mean_filter_dict["low__noised_images"][0], title="Adaptive Mean Filtered Image")

In [ ]:
def save_filtered_images_adaptive(filter_dict, images, output_dir, filter_name):
    """
    Save filtered images in a directory structure based on noise level, without kernel size.

    Parameters:
    - filter_dict: Dictionary containing filtered images organized by noise level.
    - images: List of image names (should match the indices in the filter_dict).
    - output_dir: Main directory to save the filtered images.
    - filter_name: Name of the filter for file naming.
    """
    os.makedirs(output_dir, exist_ok=True)  # Create main output directory if it doesn't exist
    
    for idx, img_name in enumerate(images):
        # Get the filtered images for each noise level
        for noise_level, img_list in filter_dict.items():
            # Create subdirectory path for each image and noise level
            subdir_path = os.path.join(output_dir, img_name, noise_level)
            os.makedirs(subdir_path, exist_ok=True)
            
            # Prepare output path
            output_path = os.path.join(subdir_path, f"{filter_name}_{img_name}_{noise_level}.png")
            
            # Handle grayscale and color images
            filtered_image = img_list[idx]
            if len(filtered_image.shape) == 3:  # Color image
                # Convert RGB to BGR if using OpenCV for color images
                filtered_image = cv.cvtColor(filtered_image, cv.COLOR_RGB2BGR)
            elif len(filtered_image.shape) == 2:  # Grayscale image
                # No conversion needed for grayscale images
                pass
            
            # Save the filtered image
            cv.imwrite(output_path, filtered_image)



In [ ]:
#store the results
output_directory = "adaptive_mean_filtered_images"
filter_name = "adaptive_mean_filter"
save_filtered_images_adaptive(adaptive_mean_filter_dict, images, output_directory, filter_name)



## 5. Adaptive Median Filtering

### What is Adaptive Median Filtering?

The Adaptive Median Filter is particularly effective for removing **salt-and-pepper noise** while 
preserving edges. Like the adaptive mean filter, it changes the kernel size based on local image 
properties but applies a median operation instead of averaging.

### Why Use an Adaptive Median Filter?

- **Noise Reduction**: It handles high-density noise regions by enlarging the kernel, which helps 
  to capture the median effectively even in noisy areas.
- **Edge Preservation**: The median operation, combined with adaptive kernel resizing, helps retain 
  sharp edges, as the median is less affected by outlier noise values than the mean.

Below, we implement the adaptive median filter, which adjusts dynamically to balance noise 
reduction with edge preservation.


In [ ]:
def adaptive_median_filtering(image, max_kernel_size=7):
    """
    Apply adaptive median filtering on an RGB image with a flexible kernel size based on local variance.
    
    Parameters:
    - image: Input image in RGB format.
    - max_kernel_size: Maximum size of the kernel (should be odd).
    
    Returns:
    - Filtered image.
    """
    if len(image.shape) == 2:
        # Single-channel grayscale image
        return _adaptive_median_filter_single_channel(image, max_kernel_size)
    elif len(image.shape) == 3:
        # RGB image, split into individual channels
        channels = cv.split(image)
        filtered_channels = []
        
        for channel in channels:
            filtered_channel = _adaptive_median_filter_single_channel(channel, max_kernel_size)
            filtered_channels.append(filtered_channel)
        
        # Merge the filtered channels back into an RGB image
        filtered_image = cv.merge(filtered_channels)
        return filtered_image
    else:
        raise ValueError("Input image must be either grayscale or RGB.")
        
def _adaptive_median_filter_single_channel(image, max_kernel_size=7):
    """
    Apply adaptive median filtering on a single-channel image.
    
    Parameters:
    - image: Input image in grayscale.
    - max_kernel_size: Maximum size of the kernel (should be odd).
    
    Returns:
    - Filtered image.
    """
    # Ensure image is float32 for calculations
    image = image.astype(np.float32)
    
    # Pad the image to handle borders
    pad_size = max_kernel_size // 2
    padded_image = cv.copyMakeBorder(image, pad_size, pad_size, pad_size, pad_size, cv.BORDER_REFLECT)
    
    # Initialize the output image
    filtered_image = np.zeros_like(image)

    # Loop over each pixel in the original image
    for i in range(pad_size, padded_image.shape[0] - pad_size):
        for j in range(pad_size, padded_image.shape[1] - pad_size):
            kernel_size = 3
            pixel_value_set = False

            while kernel_size <= max_kernel_size:
                # Extract the local region
                local_region = padded_image[i - kernel_size//2 : i + kernel_size//2 + 1, 
                                            j - kernel_size//2 : j + kernel_size//2 + 1]
                
                # Calculate median, min, and max
                median_value = np.median(local_region)
                min_value = np.min(local_region)
                max_value = np.max(local_region)
                
                A = median_value - min_value
                B = median_value - max_value

                # Step 1: Check if the median can be used as the output pixel
                if A > 0 and B < 0:
                    # Step 2: Check if the pixel should be replaced
                    if min_value < padded_image[i, j] < max_value:
                        filtered_image[i - pad_size, j - pad_size] = padded_image[i, j]
                    else:
                        filtered_image[i - pad_size, j - pad_size] = median_value
                    pixel_value_set = True
                    break
                else:
                    # Increase the kernel size
                    kernel_size += 2
            
            # If no valid median was found, set to median of the largest kernel
            if not pixel_value_set:
                filtered_image[i - pad_size, j - pad_size] = median_value

    # Convert back to uint8
    filtered_image = np.clip(filtered_image, 0, 255).astype(np.uint8)
    
    return filtered_image


In [ ]:
#we will now apply the adaptive median filtering for all noise levels
adaptive_median_filter_dict = {
    "low_sp_noised_images": {},
    "med_sp_noised_images": {},
    "high_sp_noised_images": {},
    "low_gauss_noised_images": {},
    "med_gauss_noised_images": {},
    "high_gauss_noised_images": {}
}
# Apply adaptive median filtering for each noise level
for noise_level in adaptive_mean_filter_dict.keys():
    for idx, img in enumerate(noisy_images_dict[noise_level]):
        adaptive_median_filter_dict[noise_level][idx] = adaptive_median_filtering(img, max_kernel_size=7)

In [ ]:
#save the images
output_directory = "adaptive_median_filtered_images"
filter_name = "adaptive_median_filter"
save_filtered_images_adaptive(adaptive_median_filter_dict, images, output_directory, filter_name)

In [ ]:
# the bilateral filter is a non-linear filter that preserves edges while removing noise
def apply_bilateral_filter(image, d=20, sigmaColor=75, sigmaSpace=75):
    """
    Apply bilateral filtering on a color image.
    
    Parameters:
    - image: Input image in RGB format.
    - d: Diameter of each pixel neighborhood.
    - sigmaColor: Filter sigma in the color space.
    - sigmaSpace: Filter sigma in the coordinate space.
    
    Returns:
    - Filtered image.
    """
    # Apply the bilateral filter
    filtered_image = cv.bilateralFilter(image, d,sigmaColor, sigmaSpace)
    return filtered_image

In [ ]:
#apply the bilateral filter
bilateral_filter_dict = {
    "low_sp_noised_images": {},
    "med_sp_noised_images": {},
    "high_sp_noised_images": {},
    "low_gauss_noised_images": {},
    "med_gauss_noised_images": {},
    "high_gauss_noised_images": {}
}
for noise_level in noisy_images_dict.keys():
    for idx, img in enumerate(noisy_images_dict[noise_level]):
        bilateral_filter_dict[noise_level][idx] = apply_bilateral_filter(img, d=50, sigmaColor=120, sigmaSpace=120)

In [ ]:
display_image_with_histogram(bilateral_filter_dict["med_gauss_noised_images"][2], title="Bilateral Filtered Image")

from the fly we notice how the bilateral filter is much faster than the median and the mean adaptive filters, it is also highly effective in removing gaussian noise. and we will observe how the SQNR and MSE are going to be between those filters in the results section.

In [ ]:
#save the images
output_directory = "bilateral_filtered_images"
filter_name = "bilateral_filter"
save_filtered_images_adaptive(bilateral_filter_dict, images, output_directory, filter_name)


## 4. Result Comparison and Analysis

In this final section, we visualize and compare the results of each adaptive filtering technique.

### Key Points to Consider

- **Noise Reduction**: Evaluate how effectively each filter reduces noise while maintaining essential 
  image details.
- **Edge Preservation**: Observe whether edges and important transitions remain clear and sharp.
- **Kernel Adaptability**: Assess how well each filter adapts to varying noise levels across the image.

The following visuals comparisons provide insight into the strengths and limitations of adaptive mean 
and median filtering methods.


In [ ]:
#we will construct a couple of necessary functions for the display of the images, and for obtaining the results

def calc_metrics_adaptive(filter_dict, images):
    """
    Calculate the MSE and SQNR for adaptive filters.
    
    Parameters:
    - filter_dict: Dictionary containing filtered images organized by noise level.
    - images: List of original images.
    
    Returns:
    - Dictionary containing the MSE and SQNR for each noise level.
    """
    metrics = {}
    noise_levels = list(filter_dict.keys())
    for noise_level in noise_levels:
        total_mse = 0
        total_sqnr = 0
        total_images = 0
        
        filtered_images = filter_dict[noise_level]
        for i, filtered_image in enumerate(filtered_images):
            mse_val = mse(images[i], filtered_images[i])
            sqnr_val = psnr(images[i], filtered_images[i])
            total_mse += mse_val
            total_sqnr += sqnr_val
            total_images += 1
        
        avg_mse = total_mse / total_images
        avg_sqnr = total_sqnr / total_images
        
        metrics[noise_level] = {"MSE": avg_mse, "SQNR": avg_sqnr}
    
    return metrics


In [ ]:
#we will now calculate the metrics for the adaptive filters
adaptive_mean_metrics = calc_metrics_adaptive(adaptive_mean_filter_dict, images_dict["images_np"])
adaptive_median_metrics = calc_metrics_adaptive(adaptive_median_filter_dict, images_np)
bilateral_metrics = calc_metrics_adaptive(bilateral_filter_dict, images_np)

In [ ]:
#we will print the results
print("Adaptive Mean Filter Metrics:")
for noise_level, metrics in adaptive_mean_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    print(f"MSE: {metrics['MSE']:.2f}")
    print(f"SQNR: {metrics['SQNR']:.2f}")

print("\nAdaptive Median Filter Metrics:")
for noise_level, metrics in adaptive_median_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    print(f"MSE: {metrics['MSE']:.2f}")
    print(f"SQNR: {metrics['SQNR']:.2f}")

print("\nBilateral Filter Metrics:")
for noise_level, metrics in bilateral_metrics.items():
    print(f"\nNoise Level: {noise_level}")
    print(f"MSE: {metrics['MSE']:.2f}")
    print(f"SQNR: {metrics['SQNR']:.2f}")

In [ ]:
#now a function to plot the metrics
def plot_filter_metrics_adaptive(adaptive_mean_metrics,adaptive_median_metrics, bilateral_metrics):
    """
    Plot histograms of SQNR and MSE for different adaptive filters and noise levels.
    
    Parameters:
    - adaptive_mean_metrics (dict): Dictionary containing MSE and SQNR for Adaptive Mean Filter with different noise levels.
    - adaptive_median_metrics (dict): Dictionary containing MSE and SQNR for Adaptive Median Filter with different noise levels.
    - bilateral_metrics (dict): Dictionary containing MSE and SQNR for Bilateral Filter with different noise levels.
    """
    # Prepare the data for histograms
    noise_levels = list(adaptive_mean_metrics.keys())  # Assume all dictionaries have the same noise levels

    # Create a grid of subplots: one for MSE and one for SQNR for each noise level
    num_plots = len(noise_levels)
    fig, axs = plt.subplots(num_plots, 2, figsize=(12, num_plots * 4))  # 2 columns (MSE, SQNR) for each plot

    # Loop through the noise levels and plot for each
    plot_idx = 0
    for noise_level in noise_levels:
        # Get MSE and SQNR for the Adaptive Mean, Adaptive Median, and Bilateral Filters
        adaptive_mean_mse = adaptive_mean_metrics[noise_level]["MSE"]
        adaptive_mean_sqnr = adaptive_mean_metrics[noise_level]["SQNR"]
        
        adaptive_median_mse = adaptive_median_metrics[noise_level]["MSE"]
        adaptive_median_sqnr = adaptive_median_metrics[noise_level]["SQNR"]
        
        bilateral_mse = bilateral_metrics[noise_level]["MSE"]
        bilateral_sqnr = bilateral_metrics[noise_level]["SQNR"]
        
        # Prepare data for plotting
        mse_values = [adaptive_mean_mse, adaptive_median_mse, bilateral_mse]
        sqnr_values = [adaptive_mean_sqnr, adaptive_median_sqnr, bilateral_sqnr]
        filters = ['Adaptive Mean Filter', 'Adaptive Median Filter', 'Bilateral Filter']
        
        # Plot MSE for current noise level
        axs[plot_idx, 0].bar(filters, mse_values, color=['blue', 'green', 'red'])
        axs[plot_idx, 0].set_title(f'{noise_level}')
        axs[plot_idx, 0].set_ylabel('MSE')
        
        # Plot SQNR for current noise level
        axs[plot_idx, 1].bar(filters, sqnr_values, color=['blue', 'green', 'red'])
        axs[plot_idx, 1].set_title(f'{noise_level}')
        axs[plot_idx, 1].set_ylabel('SQNR')

        plot_idx += 1  # Move to the next plot

    # Adjust layout to prevent overlapping
    plt.tight_layout()
    plt.show()

In [ ]:
plot_filter_metrics_adaptive(adaptive_mean_metrics, adaptive_median_metrics, bilateral_metrics)

# **Conclusion: Filter Performance Analysis**

After evaluating the performance of three image filtering techniques—**Bilateral Filter**, **Adaptive Median Filter**, and **Adaptive Mean Filter**—across various noise types and levels, we observed the following key results:

---

### **1. Run Time Comparison**

In terms of execution speed, the **Bilateral Filter** stands out as the fastest option by a large margin:

| **Filter Type**        | **Time (for all noise levels)** |
|------------------------|---------------------------------|
| **Adaptive Median Filter** | 8 minutes                      |
| **Adaptive Mean Filter**   | 15 minutes (slowest)           |
| **Bilateral Filter**       | **11 seconds** (fastest)       |

- **Key Insight**: The **Bilateral Filter** completes the task in a fraction of the time compared to the adaptive filters, making it ideal for real-time applications or large datasets.

---

### **2. Objective Metrics (MSE and SQNR)**

- **Salt-and-Pepper Noise**:
  - The **Adaptive Median Filter** performs excellently in removing salt-and-pepper noise, achieving the highest **SQNR** (Signal-to-Quantization Noise Ratio), and maintaining a very low **MSE** (Mean Squared Error). This indicates minimal distortion to the original image.

| **Filter Type**        | **SQNR**    | **MSE**     |
|------------------------|-------------|-------------|
| **Adaptive Median Filter** | Highest SQNR | Very Low MSE |
| **Adaptive Mean Filter**   | Moderate SQNR | Moderate MSE |
| **Bilateral Filter**       | Moderate SQNR | Moderate MSE |

- **Gaussian Noise**:
  - When dealing with **Gaussian Noise**, the **Bilateral Filter** demonstrates a surprising advantage. It not only **outperforms** the **Adaptive Median Filter** in terms of **SQNR** and **MSE**, but it also achieves a **lower MSE** while maintaining the image’s integrity more effectively.

| **Filter Type**        | **SQNR**    | **MSE**     |
|------------------------|-------------|-------------|
| **Adaptive Median Filter** | Moderate SQNR | Low MSE      |
| **Adaptive Mean Filter**   | Low SQNR     | High MSE     |
| **Bilateral Filter**       | **Highest SQNR** | **Lowest MSE** |

---

### **3. Performance Summary**

| **Noise Type**         | **Best Filter**              | **Key Strengths**                                       |
|------------------------|------------------------------|---------------------------------------------------------|
| **Salt-and-Pepper Noise** | **Adaptive Median Filter**   | Best at removing salt-and-pepper noise with minimal image distortion |
| **Gaussian Noise**       | **Bilateral Filter**         | Best at preserving image details while removing noise effectively |

---

### **4. Final Insights**

- **Bilateral Filter** has emerged as the overall winner, offering a **significant speed advantage** and **superior performance** in handling **Gaussian Noise**, all while preserving the image’s original quality.
  
- The **Adaptive Median Filter** excels in removing salt-and-pepper noise and is especially useful when noise variation is the dominant issue.

- The **Adaptive Mean Filter**, while slower, still provides a reasonable tradeoff for general noise removal tasks, though it does not outperform the Bilateral Filter in either speed or quality.

---

### **Conclusion**: The **Bilateral Filter** stands out as the most **versatile** and **efficient** filter, with excellent **noise reduction capabilities** and **fast execution times**, making it the ideal choice for most practical applications. 



### **Feedback and Final Notes**

This long but insightful experiment allowed us to explore a versatile set of filters and observe their impact on images with varying levels of complexity. Throughout the process, we gained valuable insights into how different filters perform, especially when applied to color images. 

One key takeaway is the potential to significantly speed up the filtering process by leveraging **multiprocessing** and **GPU acceleration**, along with efficient libraries like **Numba**. These tools can help optimize the performance, making the filtering tasks much faster and more efficient.

While the experience was incredibly engaging, it would have been even better if the process were a bit more concise. Nonetheless, it was a rewarding journey of learning and experimentation.

Thank you for engaging with this notebook.

**Made by**: Jehad Halahla

---
